In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score,mean_squared_error,r2_score
import seaborn as sb
import matplotlib.pyplot as plt

In [3]:
dataset = pd.read_csv('house-prices-advanced-regression-techniques/train.csv')
# dataset.head()

In [3]:
dataset = dataset.drop('Id',axis = 1)
Y = dataset['SalePrice']
dataset = dataset.drop('SalePrice',axis = 1)

In [4]:
def get_cols_with_no_nans(df,col_type):
    if (col_type == 'num'):
        predictors = df.select_dtypes(exclude=['object'])
    elif (col_type == 'no_num'):
        predictors = df.select_dtypes(include=['object'])
    elif (col_type == 'all'):
        predictors = df
    else :
        print('Error : choose a type (num, no_num, all)')
        return 0
    cols_with_no_nans = []
    for col in predictors.columns:
        if not df[col].isnull().any():
            cols_with_no_nans.append(col)
    return cols_with_no_nans

In [5]:
num_cols = get_cols_with_no_nans(dataset , 'num')
cat_cols = get_cols_with_no_nans(dataset , 'no_num')

In [6]:
print ('Number of numerical columns with no nan values :',len(num_cols))
print ('Number of nun-numerical columns with no nan values :',len(cat_cols))

('Number of numerical columns with no nan values :', 33)
('Number of nun-numerical columns with no nan values :', 27)


In [7]:
dataset[num_cols] = (dataset[num_cols] - np.mean(dataset[num_cols]))/np.std(dataset[num_cols])

In [8]:
dataset = pd.concat([dataset[num_cols],dataset[cat_cols]],axis = 1)

In [9]:
def oneHotEncode(df,colNames):
    for col in colNames:
        if( df[col].dtype == np.dtype('object')):
            dummies = pd.get_dummies(df[col],prefix=col)
            df = pd.concat([df,dummies],axis=1)

            #drop the encoded column
            df.drop([col],axis = 1 , inplace=True)
    return df
    

print('There were {} columns before encoding categorical features'.format(dataset.shape[1]))
dataset = oneHotEncode(dataset, cat_cols)
print('There are {} columns after encoding categorical features'.format(dataset.shape[1]))

There were 60 columns before encoding categorical features
There are 215 columns after encoding categorical features


In [10]:
class Neural_network(object):
    def __init__(self):
        self.input_layer_size = 215
        self.output_layer_size = 1
        self.hidden_layer_size = 100
        self.hidden_layers = 3
        self.weights = {}
        self.dweights = {}
        self.Z = {}
        self.Y_hat = np.array([])
        self.delta = {}
        
    def initialization(self):
        #Initialization of weights
        self.weights[1] = np.random.randint(low = 0,high = 1,size = (self.input_layer_size,self.hidden_layer_size),dtype = 'int')
        self.weights[self.hidden_layers+1] = np.random.randint(low = 1,high = 2,size = (self.hidden_layer_size,self.output_layer_size), dtype = 'int')
        for i in np.arange(2,self.hidden_layers+1):
            self.weights[i] = np.random.randn(self.hidden_layer_size,self.hidden_layer_size)

    
    def error(self,Y):
        y = np.array(Y)
        return 0.5*1/len(Y)*sum(((y - self.Y_hat.T)*(y-self.Y_hat.T))[0])
   
    def deltas(self,Y):
        y = np.array(Y)
        self.delta[self.hidden_layers+2] = -(y - self.Y_hat.T).T#*(self.Z[self.hidden_layers+2])
        self.dweights[self.hidden_layers+1] = np.dot(self.Z[self.hidden_layers+1].T , (self.delta[self.hidden_layers+2]))
        for i in np.arange(self.hidden_layers+1,1,-1):
            self.delta[i] =  np.dot(self.delta[i+1], self.weights[i].T)#*(self.Z[i])
            self.dweights[i-1] = np.dot(self.Z[i-1].T , self.delta[i])
    
    def forward_propagation(self, X):
        self.Z[1] = np.array(X.T[0:self.input_layer_size].T)
        for i in range(1,self.hidden_layers+2):
            self.Z[i+1] = np.dot(self.Z[i],self.weights[i])
        self.Y_hat = self.Z[self.hidden_layers+2]
        
    def gradient_descent(self,X,Y,iterations,alpha):
        for i in range(iterations):
            self.forward_propagation(X)
#             print("Error : ",self.error(Y))
            self.deltas(Y)
            for j in range(1,self.hidden_layers+2):
                self.weights[j] = self.weights[j] - alpha*self.dweights[j]
#         print("Final Error : ",self.error(Y))
        
    def predict(self,X_test):
        self.forward_propagation(X_test)
    
    def accuracy(self,Y):
        predicted = np.argmax(self.Y_hat,axis = 1)
        print("Accuracy score : ",100*(accuracy_score(predicted,np.array(Y))))
            

In [11]:
X_train = dataset.iloc[0:1000]
Y_train = Y.iloc[0:1000]
Y_train = Y_train/100000

In [12]:
k = Neural_network()
k.initialization()
k.forward_propagation(X_train)
k.gradient_descent(X_train,Y_train,100,0.00000000001)

In [13]:
X_test = dataset[1000:]
Y_test = Y[1000:]

In [17]:
k.predict(X_test)
mean_squared_error(np.array(Y_test/100000),k.Y_hat.T[0])
print(r2_score(np.array(Y_test/100000),k.Y_hat.T[0]))

0.7018822003127732


The neural network in Part1 is meant for classification and Part2 is meant for regression.
Three fundamentals design choices for neural networks are:-<br>(a) the activation function<br> (b) the loss function (i.e., how each error is going to be penalized during the optimization process)<br>(c) the regularization term (which incorporates all the a-priori information you can have on the optimal network model).

# Modifications to part-1 neural network for part-2 task are:-

## Data Processing:- 
1- Removed the columns that contains Nan value.<br>
2- Converted remaining categorical variable into dummy/indicator variables.<br>
3- Standardized the numerical attributes.

## For point (a), we generally use the same activation function as used in the prior layers (e.g. an hyperbolic tangent). The crucial point is that the output of the network does not need to be binarized to get the final output.
The output layer consist of only one node but in part-1 neural network number of nodes in output layer were equal to the number of output classes.

## Point (b) is the crucial one. In classification, we typically use the multi-class cross-entropy with a softmax layer in output, but this cannot be used for regression. In this case, a common choice is the squared error (which has a very easy derivative, but is not reliable with outliers).

## Point (c) mentioned above does not change between classification and regression.




